In [10]:
import pandas as pd 

def log_context(tx_uid,history,history_df,decisions,df_price,s_rsi_df):
    print(history)
    print(history_df)
    print(decisions)
    print(df_price)
    print(s_rsi_df)
    history_df_prime = pd.DataFrame(history)
    history_df_prime.to_csv('history_df_prime.csv')
    history_df.to_csv('history_df.csv')
    decisions_df = pd.DataFrame(decisions)
    decisions_df.to_csv('decisions_df.csv')
    df_price.to_csv('df_price.csv')
    s_rsi_df.to_csv('s_rsi_df.csv')
    print('history_df_prime',history_df_prime.shape)
    print('history_df',history_df.shape)
    print('decisions_df',decisions_df.shape)
    print('df_price',df_price.shape)
    print('s_rsi_df',s_rsi_df.shape)
    return

In [ ]:
def show_context(df):
    df_group['serie'].plot()
    return

In [11]:
import blankly
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import adfuller
import pandas_ta

import numpy as np
from scipy.stats import expon, kstest

import time
%matplotlib inline

buyc = 0 
sellc = 0

trade_time_limit = 60
trade_intensity_limit = 0.00015

def price_event(price, symbol, state: blankly.StrategyState):
    global buyc 
    global sellc


    state.variables['history'].append(price) # appends to the deque of historical prices

    """ This function will give an updated price every 15 seconds from our definition below """
    oldPrice = state.variables['history'][-1]
    oldPrice2 = state.variables['history'][-2]
    oldPrice3 = state.variables['history'][-3]
    oldPrice4 = state.variables['history'][-4]

    data = np.array(state.variables['history'])
    #print( "data len", len(data))
    datetest = pd.date_range(start='1/1/2023', periods=len(data), freq='1T').to_series()
    #use state.variables['datetime']

    variables_df = pd.DataFrame(data,index=datetest,columns=['serie'])
    series = variables_df['serie'].tail(5).pct_change().fillna(0)

    #my_rsi = float(pandas_ta.stochrsi(close=variables_df['serie'],length=20).tail(1))
    second_rsi,stochrsi_K,stochrsi_D = blankly.indicators.stochastic_rsi(state.variables['history'], period = 1440)

    lags = [1, 2, 3, 5, 10, 21, 50, 100, 200 ]
    
    q = 0.0001
    for lag in lags:
        variables_df[f'return_{lag}d'] = (variables_df['serie']
                                    .pct_change(lag)
                                    .pipe(lambda x: x.clip(lower=x.quantile(q),
                                                        upper=x.quantile(1 - q)))
                                    .add(1)
                                    .pow(1 / lag)
                                    .sub(1)
                                    )

    #condition_buy = (variables_df['return_3d'].iloc[-1] > 0 and (variables_df['return_10d'].iloc[-1] > 0) and (variables_df['return_21d'].iloc[-1] > 0) and (variables_df['return_50d'].iloc[-1] > 0))
    #condition_sell = (variables_df['return_3d'].iloc[-1] < 0 and (variables_df['return_10d'].iloc[-1] < 0) and (variables_df['return_21d'].iloc[-1] < 0) and (variables_df['return_50d'].iloc[-1] < 0))
    condition_buy = False
    condition_sell = False 
    
    df_group = variables_df.groupby(pd.Grouper(freq='20T')).agg({'return_1d':'max'}).copy()   #MEAN ? ?????
    df_group['serie'] = df_group['return_1d'].rolling(window=1).mean()

    # Generate a random dataframe with exponential distribution
    df = np.random.exponential(scale=1, size=12)
    numpy_var = df_group['serie'].iloc[-12:].to_numpy().squeeze()

    # Calculate the mean and standard deviation of the dataframe
    mean = np.mean(numpy_var)
    std = np.std(numpy_var)

    # Generate a random exponential distribution with the same mean
    exponential = expon(scale=mean)

    # Perform a Kolmogorov-Smirnov test
    p_value = kstest(numpy_var, exponential.cdf)[1]

    # Check if the p-value is less than 0.05
    if p_value < 0.05:
        #print("The data does not follow an exponential distribution.")
        condition_buy = False
        condition_sell = False 
    else:
        #print("The data follows an exponential distribution.")
        if (variables_df['return_50d'].iloc[-1] > trade_intensity_limit  ): #and  state trade_temp 
            condition_buy = True
            condition_sell = False
        elif (variables_df['return_50d'].iloc[-1] < trade_intensity_limit ):
            condition_buy = False
            condition_sell = True
        else : 
            condition_buy = False
            condition_sell = False    

 
    #condition_buy = ((indicator_7_1 or indicator_7_2 or indicator_7_3)>0.01) and ((indicator_20_1 or indicator_20_2 or indicator_20_3) >0.02) and ((indicator_50_1 or indicator_50_2 or indicator_50_3) >0.04)
    #condition_sell = ((indicator_7_1 or indicator_7_2 or indicator_7_3)<-0.01) and ((indicator_20_1 or indicator_20_2 or indicator_20_3)<-0.02) and ((indicator_50_1 or indicator_50_2 or indicator_50_3) <-0.04)
    bbands = blankly.indicators.bbands(state.variables['history'], period=20)
    
    last_10 = 10

    if (  condition_buy and stochrsi_K.iloc[-1] <= 30  and not state.variables['owns_position'] ): # (float(res_7_min['rsi'].tail(1))<60)  or second_rsi[-1] <= 50  
        #print('buy time',len(data) ,'rsi : ', stochrsi_K.iloc[-1], ' indicators: ',indicator_7_1,'-',indicator_20_1,'-',indicator_50_1 ) # float(res_7_min['rsi'].tail(1))
        print('\nbuy time',len(data) ,'rsi : ', stochrsi_K.iloc[-1], ' indicators: ',np.around(variables_df['return_3d'].iloc[-1],4),',',np.around(variables_df['return_10d'].iloc[-1],4),',',np.around(variables_df['return_21d'].iloc[-1],4) ) # float(res_7_min['rsi'].tail(1))
        buyc=buyc+1

        log_context(len(data), np.array(state.variables['history'])[-720:], variables_df.iloc[-720:][['serie','return_1d','return_50d']], state.variables['sliding_frame'][-720:],df_group['serie'].iloc[-720:],stochrsi_K.iloc[-720:])
    #if (condition_buy > 0.02 and not state.variables['owns_position']):
    #if (condition and not state.variables['owns_position']):
    #if (result[0] > 500 and not state.variables['owns_position']):
    #if ((int(bbands[0][-1] > price) + int(bbands[0][-1] > oldPrice) + int(bbands[0][-1] > oldPrice2) + int(bbands[0][-1]> oldPrice3)+ int(bbands[0][-1]> oldPrice4))>=3) and not state.variables['owns_position']:
        #buy = min(blankly.trunc(state.interface.cash / price, 3) , 0.1)
        buy = blankly.trunc(state.interface.cash / price , 4)  # 500 euro eth / size 0.25 trade  * 0.3
        #buy = int(state.interface.cash / price)
        #print('buy','----------------------',symbol, buy)
        if len(state.variables['sliding_frame'])<trade_time_limit:
            if (not (1 in state.variables['sliding_frame'])):
                state.variables['sliding_frame'].insert(1,0)
                state.interface.market_order(symbol, side='buy', size=buy)
                state.variables['owns_position'] = True
            else :
                state.variables['sliding_frame'].insert(0,0)
                state.variables['owns_position'] = True
        else:
            if not (1 in state.variables['sliding_frame']):
                state.variables['sliding_frame'].insert(1,0)
                state.interface.market_order(symbol, side='buy', size=buy)
                state.variables['sliding_frame'].pop()
                state.variables['owns_position'] = True
            else :
                state.variables['sliding_frame'].insert(0,0)
                state.variables['sliding_frame'].pop()
                state.variables['owns_position'] = True
    elif (  condition_sell and stochrsi_K.iloc[-1] >= 90 and state.variables['owns_position']): # or second_rsi[-1] >= 70 
        sellc=sellc+1
        print('\nsell time',len(data) ,'rsi : ', stochrsi_K.iloc[-1]  , ' indicators: ',np.around(variables_df['return_3d'].iloc[-1],4),',',np.around(variables_df['return_10d'].iloc[-1],4),',',np.around(variables_df['return_21d'].iloc[-1],4) )
        #print('sell time',len(data) ,'rsi : ', stochrsi_K.iloc[-1]  , ' indicators: ',indicator_7_1,'-',indicator_20_1,'-',indicator_50_1 )
    #elif (condition_sell > 0.02 and state.variables['owns_position']):
    #elif ((int(bbands[2][-1] <  price) + int(bbands[2][-1] <  oldPrice) + int(bbands[2][-1]<  oldPrice2) + int(bbands[2][-1]<  oldPrice3)+ int(bbands[2][-1]<  oldPrice4))>=3) and state.variables['owns_position']:
        #bbands[2][-1] < price and ( bbands[2][-1] < oldPrice or bbands[2][-1] < oldPrice2 or bbands[2][-1] < oldPrice3  ) and state.variables['owns_position']:
        #curr_value =  min(blankly.trunc( state.interface.account[state.base_asset].available, 3), 0.1)
        curr_value =  blankly.trunc( state.interface.account[state.base_asset].available, 4) #1.3939  1.394
        #curr_value = int(state.interface.account[state.base_asset].available)
        #print('sell','----------------------',symbol, curr_value)
        if len(state.variables['sliding_frame'])<trade_time_limit:
            if not (-1 in state.variables['sliding_frame']):
                state.variables['sliding_frame'].insert(-1,0)
                state.interface.market_order(symbol, side='sell', size=curr_value)
                state.variables['owns_position'] = False
            else :
                state.variables['sliding_frame'].insert(0,0)
                state.variables['owns_position'] = False
        else:
            if not (-1 in state.variables['sliding_frame']):
                state.variables['sliding_frame'].insert(-1,0)
                state.interface.market_order(symbol, side='sell', size=curr_value)
                state.variables['sliding_frame'].pop()
                state.variables['owns_position'] = False
            else :
                state.variables['sliding_frame'].insert(0,0)
                state.variables['sliding_frame'].pop()
                state.variables['owns_position'] = False
    else:
        if len(state.variables['sliding_frame'])<trade_time_limit:
            state.variables['sliding_frame'].insert(0,0)
        else:
            state.variables['sliding_frame'].insert(0,0)
            state.variables['sliding_frame'].pop()
    return


def init(symbol, state: blankly.StrategyState):
    # Download price data to give context to the algo
    state.variables['history'] = state.interface.history(symbol, to=13000, return_as='deque')['close']
    state.variables['owns_position'] = False
    state.variables['sliding_frame'] = []

In [12]:
from lppls import lppls, data_loader
import numpy as np
import pandas as pd
from datetime import datetime as dt
import pandas as pd

%matplotlib inline

if __name__ == "__main__":
    # Authenticate Binance strategy
    exchange = blankly.Binance()

    # Use our strategy helper on Binance
    strategy = blankly.Strategy(exchange)

    # Run the price event function every time we check for a new price - by default that is 15 seconds 
    # zen , bnb
    strategy.add_price_event(price_event, symbol='ETH-USDT', resolution='1m', init=init)   # resolution='1d',

    # Start the strategy. This will begin each of the price event ticks
    # strategy.start()
    # Or backtest using this
    results = strategy.backtest(to='12d', initial_values={'USDT': 10000}, settings_path='./backtest.json') #Backtest one year starting with $10,000
    results.get_account_history().to_csv("get_account_history.csv")
    results.get_returns().to_csv("get_returns.csv")
    results.get_resampled_account().to_csv("get_resampled_account.csv")
    pd.DataFrame(results.get_metrics()).to_csv("get_metrics.csv")

INFO: No portfolio name to load specified, defaulting to the first in the file: (another cool portfolio). This is fine if there is only one portfolio in use.


No cached data found for ETH-USDT from: 1683535324 to 1683535417.0416074 at a resolution of 60 seconds.

Backtesting...
Progress: [----------] 3.46%  
buy time 2882 rsi :  13.02  indicators:  0.0009 , 0.0002 , 0.0005
[1894.82 1895.95 1896.25 1895.9  1896.36 1898.04 1894.91 1895.77 1895.33
 1894.72 1894.01 1893.7  1895.89 1896.21 1895.72 1898.23 1900.34 1897.54
 1898.76 1900.19 1903.01 1904.47 1904.8  1903.5  1906.09 1906.12 1904.63
 1906.15 1905.68 1905.34 1905.7  1908.41 1911.32 1913.8  1917.32 1917.98
 1915.27 1915.34 1914.63 1917.36 1917.   1916.99 1917.7  1918.16 1918.56
 1916.65 1914.58 1913.59 1913.89 1913.87 1917.2  1915.77 1916.05 1917.42
 1917.85 1919.2  1917.49 1918.36 1916.75 1918.08 1915.   1913.14 1913.93
 1914.19 1913.66 1913.46 1912.21 1914.17 1912.93 1912.   1911.28 1911.4
 1911.8  1911.33 1910.52 1910.77 1911.97 1913.   1912.52 1911.   1911.57
 1911.56 1911.19 1912.13 1913.96 1914.12 1913.19 1913.99 1914.5  1915.03
 1915.25 1914.74 1914.26 1914.2  1914.72 1914.86 1916.

Traceback (most recent call last):
  File "c:\Users\Sim\miniconda3\lib\site-packages\blankly\frameworks\strategy\strategy.py", line 127, in rest_event
    callback(*args)
  File "C:\Users\Sim\AppData\Local\Temp\ipykernel_14192\2061553763.py", line 107, in price_event
    log_context(len(data), np.array(state.variables['history'])[-720:], variables_df.iloc[-720:][['serie','return_1d','return_50d']], state.variables['sliding_frame'][-720:],df_group['serie'].iloc[-720:],stochrsi_K.iloc[-720:])
  File "C:\Users\Sim\AppData\Local\Temp\ipykernel_14192\3791833322.py", line 18, in log_context
    print('decisions',decisions.size)
AttributeError: 'list' object has no attribute 'size'
Traceback (most recent call last):
  File "c:\Users\Sim\miniconda3\lib\site-packages\blankly\frameworks\strategy\strategy.py", line 127, in rest_event
    callback(*args)
  File "C:\Users\Sim\AppData\Local\Temp\ipykernel_14192\2061553763.py", line 107, in price_event
    log_context(len(data), np.array(state.variab

history_df_prime 720
history_df 720

buy time 2888 rsi :  13.09  indicators:  -0.0002 , 0.0 , 0.0005
[1894.91 1895.77 1895.33 1894.72 1894.01 1893.7  1895.89 1896.21 1895.72
 1898.23 1900.34 1897.54 1898.76 1900.19 1903.01 1904.47 1904.8  1903.5
 1906.09 1906.12 1904.63 1906.15 1905.68 1905.34 1905.7  1908.41 1911.32
 1913.8  1917.32 1917.98 1915.27 1915.34 1914.63 1917.36 1917.   1916.99
 1917.7  1918.16 1918.56 1916.65 1914.58 1913.59 1913.89 1913.87 1917.2
 1915.77 1916.05 1917.42 1917.85 1919.2  1917.49 1918.36 1916.75 1918.08
 1915.   1913.14 1913.93 1914.19 1913.66 1913.46 1912.21 1914.17 1912.93
 1912.   1911.28 1911.4  1911.8  1911.33 1910.52 1910.77 1911.97 1913.
 1912.52 1911.   1911.57 1911.56 1911.19 1912.13 1913.96 1914.12 1913.19
 1913.99 1914.5  1915.03 1915.25 1914.74 1914.26 1914.2  1914.72 1914.86
 1916.63 1914.86 1915.04 1914.09 1914.18 1913.62 1914.15 1914.17 1915.79
 1916.68 1914.76 1913.66 1913.89 1912.83 1913.39 1914.58 1914.55 1915.62
 1915.05 1915.02 1915.16 19

Traceback (most recent call last):
  File "c:\Users\Sim\miniconda3\lib\site-packages\blankly\frameworks\strategy\strategy.py", line 127, in rest_event
    callback(*args)
  File "C:\Users\Sim\AppData\Local\Temp\ipykernel_14192\2061553763.py", line 107, in price_event
    log_context(len(data), np.array(state.variables['history'])[-720:], variables_df.iloc[-720:][['serie','return_1d','return_50d']], state.variables['sliding_frame'][-720:],df_group['serie'].iloc[-720:],stochrsi_K.iloc[-720:])
  File "C:\Users\Sim\AppData\Local\Temp\ipykernel_14192\3791833322.py", line 18, in log_context
    print('decisions',decisions.size)
AttributeError: 'list' object has no attribute 'size'
Traceback (most recent call last):
  File "c:\Users\Sim\miniconda3\lib\site-packages\blankly\frameworks\strategy\strategy.py", line 127, in rest_event
    callback(*args)
  File "C:\Users\Sim\AppData\Local\Temp\ipykernel_14192\2061553763.py", line 107, in price_event
    log_context(len(data), np.array(state.variab

...

history_df_prime 720
history_df 720

buy time 3065 rsi :  28.5  indicators:  0.0001 , -0.0003 , -0.0
[1956.5  1957.08 1957.65 1957.61 1956.81 1954.37 1954.95 1953.34 1953.23
 1955.14 1955.6  1953.86 1952.79 1953.94 1954.3  1954.46 1954.68 1955.38
 1952.79 1953.18 1952.38 1951.39 1947.03 1947.52 1950.71 1950.05 1950.
 1949.12 1946.68 1944.16 1945.21 1945.06 1945.43 1944.07 1947.85 1947.18
 1949.49 1951.55 1951.39 1952.44 1952.05 1953.18 1953.16 1953.84 1952.95
 1951.77 1952.91 1954.75 1954.2  1954.1  1953.35 1953.6  1953.5  1950.86
 1952.04 1953.68 1951.13 1950.01 1950.23 1950.72 1951.53 1949.58 1948.42
 1947.72 1947.52 1947.65 1945.77 1940.73 1938.41 1939.2  1938.52 1936.13
 1934.59 1935.86 1938.79 1939.22 1938.18 1939.18 1939.71 1940.45 1939.54
 1941.52 1940.8  1939.13 1936.99 1936.89 1937.17 1938.37 1939.   1938.44
 1937.71 1940.34 1940.63 1940.62 1942.39 1942.17 1941.67 1943.09 1943.52
 1942.11 1944.7  1943.83 1944.34 1944.24 1945.36 1946.77 1947.02 1946.24
 1946.05 1946.78 1947.68 

Traceback (most recent call last):
  File "c:\Users\Sim\miniconda3\lib\site-packages\blankly\frameworks\strategy\strategy.py", line 127, in rest_event
    callback(*args)
  File "C:\Users\Sim\AppData\Local\Temp\ipykernel_14192\2061553763.py", line 107, in price_event
    log_context(len(data), np.array(state.variables['history'])[-720:], variables_df.iloc[-720:][['serie','return_1d','return_50d']], state.variables['sliding_frame'][-720:],df_group['serie'].iloc[-720:],stochrsi_K.iloc[-720:])
  File "C:\Users\Sim\AppData\Local\Temp\ipykernel_14192\3791833322.py", line 18, in log_context
    print('decisions',decisions.size)
AttributeError: 'list' object has no attribute 'size'
Traceback (most recent call last):
  File "c:\Users\Sim\miniconda3\lib\site-packages\blankly\frameworks\strategy\strategy.py", line 127, in rest_event
    callback(*args)
  File "C:\Users\Sim\AppData\Local\Temp\ipykernel_14192\2061553763.py", line 107, in price_event
    log_context(len(data), np.array(state.variab


buy time 3129 rsi :  27.17  indicators:  0.0022 , 0.0006 , 0.0005
[1947.52 1947.65 1945.77 1940.73 1938.41 1939.2  1938.52 1936.13 1934.59
 1935.86 1938.79 1939.22 1938.18 1939.18 1939.71 1940.45 1939.54 1941.52
 1940.8  1939.13 1936.99 1936.89 1937.17 1938.37 1939.   1938.44 1937.71
 1940.34 1940.63 1940.62 1942.39 1942.17 1941.67 1943.09 1943.52 1942.11
 1944.7  1943.83 1944.34 1944.24 1945.36 1946.77 1947.02 1946.24 1946.05
 1946.78 1947.68 1947.79 1946.26 1945.46 1946.37 1945.78 1946.3  1946.64
 1946.05 1947.75 1946.97 1947.36 1947.63 1949.57 1947.94 1948.16 1948.94
 1948.07 1947.69 1949.18 1948.81 1947.53 1946.16 1947.14 1946.47 1946.01
 1946.7  1946.69 1946.18 1946.99 1946.2  1947.48 1948.07 1946.72 1948.59
 1946.36 1947.46 1947.5  1948.31 1949.44 1949.74 1952.01 1952.81 1951.07
 1951.59 1952.42 1951.39 1953.21 1953.22 1953.7  1952.76 1953.08 1952.56
 1952.68 1953.6  1952.83 1952.62 1952.14 1953.91 1953.92 1954.45 1953.26
 1952.88 1953.13 1954.25 1957.29 1958.14 1957.18 1956.37 

Traceback (most recent call last):
  File "c:\Users\Sim\miniconda3\lib\site-packages\blankly\frameworks\strategy\strategy.py", line 127, in rest_event
    callback(*args)
  File "C:\Users\Sim\AppData\Local\Temp\ipykernel_14192\2061553763.py", line 107, in price_event
    log_context(len(data), np.array(state.variables['history'])[-720:], variables_df.iloc[-720:][['serie','return_1d','return_50d']], state.variables['sliding_frame'][-720:],df_group['serie'].iloc[-720:],stochrsi_K.iloc[-720:])
  File "C:\Users\Sim\AppData\Local\Temp\ipykernel_14192\3791833322.py", line 18, in log_context
    print('decisions',decisions.size)
AttributeError: 'list' object has no attribute 'size'


Progress: [#---------] 6.93% 